<a href="https://colab.research.google.com/github/saiKelkar/Computer-Vision-Deep-Learning/blob/main/SVM_without_regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Multiclass SVM without regularization
def L_i(x, y, W):
    """
    unvectorized version. Compute the multiclass SVM Loss for a single example (x, y)
    - x is a column vector representing an image (e.g. 3073 x 1 in CIFAR-10)
        with an appended bias dimension in the 3073-rd position (i.e. bias trick)
    - y is an integer giving index of correct class (e.g. between 0 and 9 in CIFAR-10)
    - W is a weight matrix (e.g. 10 x 3073 in CIFAR-10)
    """
    delta = 1.0
    # scores becomes of size 10 x 1, the scores for each class
    scores = W.dot(x)
    correct_class_score = scores[y]
    D = W.shape[0]
    loss_i = 0.0

    # iterate over all wrong classes
    for j in range(D):
        if j == y:
            # skip for the true class to only loop over incorrect classes
            continue
        # accumulate loss for the i-th example
        loss_i += max(0, scores[j] - correct_class_score + delta)
    return loss_i

In [3]:
import numpy as np

def L_i_vectorized(x, y, W):
    """
    A faster half-vectorized implementation. half-vectorized
    refers to the fact that for a single example the implementation contains
    no for loops, but there is still one loop over the examples (outside this function)
    """
    delta = 1.0
    scores = W.dot(x)
    # compute the margins for all classes in one vector operation
    margins = np.maximum(0, scores - scores[y] + delta)
    # do not count the correct class
    margins[y] = 0
    loss_i = np.sum(margins)
    return loss_i

In [4]:
def svm_loss_vectorized(X, y, W):
    """
    Fully vectorized loss computation for all training examples.

    Inputs:
    X: input data of shape (N, D) where each row is a D-dimensional input example
    y: Vector of labels of shape (N,), with values in [0, C)
    W: Weight matrix of shape (C, D)

    Returns:
    loss: scalar value of the total loss
    """

    delta = 1.0
    scores = X.dot(W.T)
    correct_class_scores = scores[np.arange(X.shape[0]), y]

    # Compute margins and apply hinge loss
    margins = np.maximum(0, scores - correct_class_scores[:, np.newaxis] + delta)
    margins[np.arange(X.shape[0]), y] = 0

    loss = np.sum(margins) / X.shape[0]
    return loss